In [2]:
import re
import glob
import os

import pandas as pd 

import pylint.lint
import nbformat

import sqlite3

In [42]:
! pip install nbformat 

Defaulting to user installation because normal site-packages is not writeable


# Data Directory Setup

In this study, data are split into 10 parts to handle them easily.</br> Use the directory pathonly if the data are gathered in a single directory.

In [3]:
# pylint_txt_path = ['data_directory']
pylint_txt_path = ['nb_data/part_0/KT_dataset/','nb_data/part_1/KT_dataset/','nb_data/part_2/KT_dataset/',
                   'nb_data/part_3/KT_dataset/','nb_data/part_4/KT_dataset/','nb_data/part_5/KT_dataset/',
                   'nb_data/part_6/KT_dataset/','nb_data/part_7/KT_dataset/','nb_data/part_8/KT_dataset/',
                   'nb_data/part_9/KT_dataset/'] 

output_path="result/all_pylint_nb_text/"

In [4]:
def get_list_of_filename(path, extension):
    notebook_filenames_list = glob.glob(os.path.join(path, extension))
    
    print(path, str(len(notebook_filenames_list)))
    return notebook_filenames_list

# Database SQlite Connection

## Database Function

In [4]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print( 'Connection established ', db_file)
        return conn
    
    except Error as e:
        print(e)

In [5]:
def close_connection(con):
    try:
        con.close()
        print ('Connection was closed now ', con)
    except Exception as e:
        print ('Connection invalid ', e)

In [6]:
def db_table_create(connection_obj):
    # Connecting to sqlite
    # connection object

    # cursor object
    cursor_obj = connection_obj.cursor()

    # Drop the table if already exists.
    cursor_obj.execute("DROP TABLE IF EXISTS NBISSUE")
    cursor_obj.execute("DROP TABLE IF EXISTS NBRATING")
    cursor_obj.execute("DROP TABLE IF EXISTS NBFILES")

    # Creating table
    table_nbissue = """ CREATE TABLE NBISSUE (
                Filename VARCHAR(5000) NOT NULL,
                CellNo INT,
                Line INT,
                Issue VARCHAR(10)
            ); """

    table_nbratings = """ CREATE TABLE NBRATING (
            Filename VARCHAR(5000) NOT NULL,
            Rating REAL
        ); """

    table_nbfiles = """ CREATE TABLE NBFILES (
        Filename VARCHAR(5000) NOT NULL,
        Type VARCHAR(10)
    ); """
    
    cursor_obj.execute(table_nbissue)
    cursor_obj.execute(table_nbratings)
    cursor_obj.execute(table_nbfiles)
    

    print("Table is Ready")
    connection_obj.commit()
    
    return cursor_obj

In [7]:
def data_insert_nbrating(connection, Filename, rating):
    cursor = connection.cursor()

    # Queries to INSERT records.    
    cursor.execute("insert into NBRATING (Filename, Rating) values (?, ?)",
            (Filename, rating))

    # Commit your changes in the database    
    connection.commit()

In [8]:
def data_insert_nbissue(connection, Filename, cell_no, line, issue):
    cursor = connection.cursor()

    # Queries to INSERT records.    
    cursor.execute("insert into NBISSUE (Filename, CellNo, Line, Issue) values (?, ?, ?, ?)",
            (Filename, cell_no, line, issue))

    # Commit your changes in the database    
    connection.commit()

In [9]:
def data_insert_nbfilenames(connection, Filename, mlnonmltype):
    cursor = connection.cursor()

    # Queries to INSERT records.    
    cursor.execute("insert into NBFILES (Filename, Type) values (?, ?)",
            (Filename, mlnonmltype))

    # Commit your changes in the database    
    connection.commit()

In [10]:
def display_data(conn, tablename):
    
    cursor = conn.cursor()
    # Display data inserted
    
    if (tablename == 'NBFILES'):
        print("Data Display for the table: NBFILES ")

        data=cursor.execute('''SELECT * FROM NBFILES''')
        for row in data:
            print(row)
        
    elif (tablename == 'NBRATING'):
        print("Data Display for the table: NBRATING ")

        data=cursor.execute('''SELECT * FROM NBRATING''')
        for row in data:
            print(row)
   
    elif   (tablename == 'NBISSUE'):  
        print("Data Display for the table: NBISSUE ")

        data=cursor.execute('''SELECT * FROM NBISSUE''')
        for row in data:
            print(row)
    
    conn.commit()

In [11]:
def count_data(conn, table):
    cursor = conn.cursor()
    
    rowsQuery = "SELECT Count() FROM %s" % table
    cursor.execute(rowsQuery)
    numberOfRows = cursor.fetchone()[0]
    print (table, numberOfRows)
    
    conn.commit()

In [12]:
def updateTable(connection):
    cursor = connection.cursor()
    query = "UPDATE NBISSUE SET FileID = FileID - 1"
    cursor.execute(query)
    
    # Commit your changes in the database    
    connection.commit()

## Database Execution 

<b>Please change the database filename </b> if you want to regenerate all the tables again. If you did not change the db filename and try to execute the rest of the code, existing data will be erased and new data start overriting there. 
</br>If you want to use the existing results, <b>please keep the db filename as it is</b>. 

In [13]:

connection = create_connection(r"data/nb_pylint_result.db")
# db_table_create(connection)


Connection established  data/nb_pylint_result.db


In [14]:
# display_data(connection, 'NBRATING')
# display_data(connection, 'NBFILES')

count_data(connection, 'NBRATING')
# count_data(connection, 'NBFILES')

NBRATING 246355


In [40]:
close_connection(connection)

Connection was closed now  <sqlite3.Connection object at 0x7fc2709466c0>


# Setup ML and Non-ML Notebook Filenames

In [15]:
ml_filenames = pd.read_csv("data/ml_filename_v1.csv")
nonml_filenames = pd.read_csv("data/nonml_filename_v1.csv")
print(len(ml_filenames), len(nonml_filenames))
nonml_filenames['Filename']

171139 69156


0                     CVxTz_audio-data-augmentation.py
1        a01653716_hogar-ingresostrimestrales-enigh.py
2                   a03102030_eda-for-covid-19-data.py
3                      a03102030_eda-for-crime-data.py
4                 a03102030_eda-for-googleplay-data.py
                             ...                      
69151                zzy990106_single-roberta-large.py
69152                     zzy990106_truncate-errors.py
69153                 zzylordreturn_icu-requirement.py
69154                              zzzttt_newsubmit.py
69155                      zzzzzfy_kernel6464812e2f.py
Name: Filename, Length: 69156, dtype: object

In [17]:
filenames_for_issues, code_cells, line_numbers, issue_codes = [], [], [], []
filenames_for_rating, ratings = [], []
outputs_files = []


# Store Data to SQLite Database

Since I split the entire data into 10 parts for better handling, I processed the Pylint output part by part [ 0 to 9].  
</br> If you have enough resources to keep the entire Notebook dataset in a single execution, please update the for loop here. The rest of the code will remain the same. 

In [38]:
file_count = 19808

In [32]:
for nb_path in pylint_txt_path[6:]:
    nb_filenames = get_list_of_filename(nb_path, "*_pylintnb.txt")
    
    for file_name in nb_filenames:
        
    
#     data insert to nb filename and type (ml or nonml)
        nb_filename_for_type_check = file_name.split('/')[-1].split('_pylintnb.txt')[0]
        nb_filename_original = file_name.replace('_pylintnb.txt', '.ipynb')
        nb_type = 'ml'
        if (nonml_filenames[nonml_filenames['Filename'].str.contains(nb_filename_for_type_check, regex=True)].any(axis=None)):
            nb_type = 'nonml'
        data_insert_nbfilenames(connection, nb_filename_original, nb_type)
        # print (nb_filename_original)
            
        with open(file_name) as fp:
            lines_pylint = fp.readlines()
            
        for line in lines_pylint:
            try:
                if line.startswith('nb_data/'):
                    # print (line)
                    # filename = line.split(':')[0]
                    cell = line.split(':')[1].split('_')[1]
                    lineNo = line.split(':')[2]
                    issue = line.split(':')[4].strip()

                    data_insert_nbissue(connection, nb_filename_original, cell, lineNo, issue)
                    

                if line.startswith('Your code has been'):
                    rating = line.split(' ')[-1].split('/')[0]

                    data_insert_nbrating(connection, nb_filename_original, rating)


            except Exception as e:
                with open("exception_log.txt", "a") as file_object:
                    write_message = str(nb_filename_original) + " : " + str(e) + "\n"
                    file_object.write(write_message)


nb_data/part_6/KT_dataset/ 24876
nb_data/part_7/KT_dataset/ 24876
nb_data/part_8/KT_dataset/ 24876
nb_data/part_9/KT_dataset/ 24877


--- NB Analysis </br>
Total 248761

In [34]:
print ('done')

done
